In [1]:
import xarray as xr
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import cartopy.crs as crs
import cartopy.feature as cfeature
import requests
import os,sys
from mpl_toolkits.basemap import Basemap, cm, shiftgrid, addcyclic
import glob
from datetime import datetime
import regionmask
import geopandas as gpd
from shapely.geometry import mapping
import pickle
import copy

ERROR 1: PROJ: proj_create_from_database: Open of /home/tiro/miniconda3/envs/my_env/share/proj failed


In [2]:
#AO Angola 
#BC Botswana 
#CF Congo (Brazzaville) 
#CG Congo (Kinshasa)
#LT Lesotho 
#MA Madagascar 
#MI Malawi
#MP Mauritius 
#MZ Mozambique 
#SE Seychelles 
#SF South Africa
#TZ Tanzania 
#WA Namibia 
#WZ Swaziland 
#ZA Zambia 
#ZI Zimbabwe 

In [3]:
# Cities in Southern Africa

#Country City lat lon
#Angola Luanda -8.871229, 13.304480
#Botswana Gaborone -24.634432, 25.915398
#Botswana Francistown -21.186244, 27.496567
#DRC Kinsasha -4.318447, 15.272359
#Lesotho Maseru -29.344155, 27.492979
#Madagascar Antananarivo -18.887962, 47.514284
#Malawi Blantye -15.769949, 35.019252
#Malawi Lilongwe -13.963143, 33.776700
#Mauritius Port Louis -20.163139, 57.503299
#Mozambique Maputo -25.924859, 32.571495
#Mozambique Beira -19.822438, 34.862059
#Seychelles Victoria -4.621871, 55.451806
#South Africa Johannesburg -26.197445, 28.032834
#South Africa Cape Town -33.928441, 18.426198
#South Africa Bloemfontein -29.120085, 26.224407
#Tanzania Dodoma -6.168677, 35.745462
#Tanzania Dar es Salaam -6.815411, 39.260712
#Namibia Windhoek -22.552644, 17.071494
#Eswatini Manzini -26.497813, 31.360386
#Zambia Lusaka -15.398946, 28.314912
#Zambia Kasama -10.212382, 31.178276
#Zimbabwe Bulawayo -20.156086, 28.582755
#Zimbabwe Harare -17.830750, 31.051156

In [4]:
# Cities in Southern Africa
#Country City lat lon
#Angola Luanda -8.871229, 13.304480
#Botswana Gaborone -24.634432, 25.915398
#Botswana Francistown -21.186244, 27.496567
#DRC Kinsasha -4.318447, 15.272359
#Lesotho Maseru -29.344155, 27.492979
#Madagascar Antananarivo -18.887962, 47.514284
#Malawi Blantye -15.769949, 35.019252
#Malawi Lilongwe -13.963143, 33.776700
#Mauritius Port Louis -20.163139, 57.503299
#Mozambique Maputo -25.924859, 32.571495
#Mozambique Beira -19.822438, 34.862059
#Seychelles Victoria -4.621871, 55.451806
#South Africa Johannesburg -26.197445, 28.032834
#South Africa Cape Town -33.928441, 18.426198
#South Africa Bloemfontein -29.120085, 26.224407
#Tanzania Dodoma -6.168677, 35.745462
#Tanzania Dar es Salaam -6.815411, 39.260712
#Namibia Windhoek -22.552644, 17.071494
#Eswatini Manzini -26.497813, 31.360386
#Zambia Lusaka -15.398946, 28.314912
#Zambia Kasama -10.212382, 31.178276
#Zimbabwe Bulawayo -20.156086, 28.582755
#Zimbabwe Harare -17.830750, 31.051156

In [21]:
def get_station_point(file_xr, station_coords, var):    
    stations = {}
    #for i in np.arange(len(station_coords)):
    a = station_coords.get(['station', 'lat', 'lon']).loc()[0]
    data_point = file_xr[var].sel(latitude = a.lat, longitude = a.lon, method="nearest")
    data_point = data_point.resample(time = 'D').sum()
    #print(data_point['time'])
    data_stations = pd.DataFrame(data_point['time'].values)
    data_stations.columns = ['time']

    for i in np.arange(len(station_coords)):
            a = station_coords.get(['station', 'lat', 'lon']).loc()[i]
            data_point = file_xr[var].sel(latitude = a.lat, longitude = a.lon, method="nearest")
            #print(data_point.latitude, data_point.longitude)
            data_point = data_point.resample(time = 'D').sum()
            data_dt = data_point.to_dataframe()[var]
            data_dt = pd.DataFrame(pd.DataFrame(data_dt).reset_index()[var])
            data_dt.columns = [a.station]
            data_stations=pd.concat([data_stations,data_dt], axis=1)

    data_stations = data_stations.set_index('time')

    return data_stations

In [7]:
root_path = '/media/tiro/nkemelang/atkins/activities/activity_2.1b/data/'

file_path_rea = root_path+'rea*/*/*/*/*/*/sadc/'
file_path_obs = root_path+'obs*/*/*/*/*/*/sadc/'

files_rea = glob.glob(file_path_rea+'*era5.nc')
files_obs = glob.glob(file_path_obs+'*era5.nc')

cities = pd.read_csv('../data/Southern_Africa_cities.csv')

station_coords = cities.get(['Country','City', 'lat', 'lon']).reset_index()
station_coords.columns = ['index', 'country', 'station', 'lat', 'lon']


In [8]:
cities.head()

,Country,City,lat,lon
0,Angola,Luanda,-8.871229,13.304480
1,Botswana,Gaborone,-24.634432,25.915398
2,Botswana,Francistown,-21.186244,27.496567
3,DRC,Kinsasha,-4.318447,15.272359
4,Lesotho,Maseru,-29.344155,27.492979


In [22]:
## Ran

cities_1 = np.unique(station_coords.station)
#cities_1 = ['Gaborone']
for city in cities_1:
    to_subset = station_coords[station_coords['station'] == city]
    to_subset = to_subset.reset_index()
    #observed_keys = []
    #observed = {}
    i=0
    for ifile in files_obs: 
        i=i+1
        observed_keys = []
        observed = {}
        var = 'pr'
        file_split=ifile.split("/")
        filename=file_split[-1]
        filename_split=filename.split("_")
        filename_base=filename.split(".")[0]
        institute=filename_split[2]  
        dataset=filename_split[3]
        #print([institute, dataset])
        if dataset == 'mswep-v280':
            var = 'precipitation'
        #print(var)
        file_xr = xr.open_dataset(ifile)
        key_name = 'results_'+dataset 
        observed_keys.append(key_name)
        data_stations = get_station_point(file_xr=file_xr, station_coords=to_subset, var = var)
        observed[key_name] = copy.deepcopy(data_stations)

        with open('../data/pickles_py/point/cities/day/1_observed_day_'+institute+'_'+dataset+'_'+city+'.pickle', 'wb') as handle:
            pickle.dump(observed, handle, protocol=pickle.HIGHEST_PROTOCOL)


<xarray.DataArray 'latitude' ()>
array(-19., dtype=float32)
Coordinates:
    longitude  float32 47.5
    latitude   float32 -19.0
Attributes:
    standard_name:  latitude
    long_name:      latitude
    units:          degrees_north
    axis:           Y <xarray.DataArray 'longitude' ()>
array(47.5, dtype=float32)
Coordinates:
    longitude  float32 47.5
    latitude   float32 -19.0
Attributes:
    standard_name:  longitude
    long_name:      longitude
    units:          degrees_east
    axis:           X
<xarray.DataArray 'latitude' ()>
array(-19., dtype=float32)
Coordinates:
    longitude  float32 47.5
    latitude   float32 -19.0
Attributes:
    standard_name:  latitude
    long_name:      latitude
    units:          degrees_north
    axis:           Y <xarray.DataArray 'longitude' ()>
array(47.5, dtype=float32)
Coordinates:
    longitude  float32 47.5
    latitude   float32 -19.0
Attributes:
    standard_name:  longitude
    long_name:      longitude
    units:          degrees

<xarray.DataArray 'latitude' ()>
array(-15.75, dtype=float32)
Coordinates:
    longitude  float32 35.0
    latitude   float32 -15.75
Attributes:
    standard_name:  latitude
    long_name:      latitude
    units:          degrees_north
    axis:           Y <xarray.DataArray 'longitude' ()>
array(35., dtype=float32)
Coordinates:
    longitude  float32 35.0
    latitude   float32 -15.75
Attributes:
    standard_name:  longitude
    long_name:      longitude
    units:          degrees_east
    axis:           X
<xarray.DataArray 'latitude' ()>
array(-15.75, dtype=float32)
Coordinates:
    longitude  float32 35.0
    latitude   float32 -15.75
Attributes:
    standard_name:  latitude
    long_name:      latitude
    units:          degrees_north
    axis:           Y <xarray.DataArray 'longitude' ()>
array(35., dtype=float32)
Coordinates:
    longitude  float32 35.0
    latitude   float32 -15.75
Attributes:
    standard_name:  longitude
    long_name:      longitude
    units:          d

<xarray.DataArray 'latitude' ()>
array(-20.25, dtype=float32)
Coordinates:
    longitude  float32 28.5
    latitude   float32 -20.25
Attributes:
    standard_name:  latitude
    long_name:      latitude
    units:          degrees_north
    axis:           Y <xarray.DataArray 'longitude' ()>
array(28.5, dtype=float32)
Coordinates:
    longitude  float32 28.5
    latitude   float32 -20.25
Attributes:
    standard_name:  longitude
    long_name:      longitude
    units:          degrees_east
    axis:           X
<xarray.DataArray 'latitude' ()>
array(-20.25, dtype=float32)
Coordinates:
    longitude  float32 28.5
    latitude   float32 -20.25
Attributes:
    standard_name:  latitude
    long_name:      latitude
    units:          degrees_north
    axis:           Y <xarray.DataArray 'longitude' ()>
array(28.5, dtype=float32)
Coordinates:
    longitude  float32 28.5
    latitude   float32 -20.25
Attributes:
    standard_name:  longitude
    long_name:      longitude
    units:         

<xarray.DataArray 'latitude' ()>
array(-6.75, dtype=float32)
Coordinates:
    longitude  float32 39.25
    latitude   float32 -6.75
Attributes:
    standard_name:  latitude
    long_name:      latitude
    units:          degrees_north
    axis:           Y <xarray.DataArray 'longitude' ()>
array(39.25, dtype=float32)
Coordinates:
    longitude  float32 39.25
    latitude   float32 -6.75
Attributes:
    standard_name:  longitude
    long_name:      longitude
    units:          degrees_east
    axis:           X
<xarray.DataArray 'latitude' ()>
array(-6.25, dtype=float32)
Coordinates:
    longitude  float32 35.75
    latitude   float32 -6.25
Attributes:
    standard_name:  latitude
    long_name:      latitude
    units:          degrees_north
    axis:           Y <xarray.DataArray 'longitude' ()>
array(35.75, dtype=float32)
Coordinates:
    longitude  float32 35.75
    latitude   float32 -6.25
Attributes:
    standard_name:  longitude
    long_name:      longitude
    units:         

<xarray.DataArray 'latitude' ()>
array(-24.75, dtype=float32)
Coordinates:
    longitude  float32 26.0
    latitude   float32 -24.75
Attributes:
    standard_name:  latitude
    long_name:      latitude
    units:          degrees_north
    axis:           Y <xarray.DataArray 'longitude' ()>
array(26., dtype=float32)
Coordinates:
    longitude  float32 26.0
    latitude   float32 -24.75
Attributes:
    standard_name:  longitude
    long_name:      longitude
    units:          degrees_east
    axis:           X
<xarray.DataArray 'latitude' ()>
array(-24.75, dtype=float32)
Coordinates:
    longitude  float32 26.0
    latitude   float32 -24.75
Attributes:
    standard_name:  latitude
    long_name:      latitude
    units:          degrees_north
    axis:           Y <xarray.DataArray 'longitude' ()>
array(26., dtype=float32)
Coordinates:
    longitude  float32 26.0
    latitude   float32 -24.75
Attributes:
    standard_name:  longitude
    long_name:      longitude
    units:          d

...

<xarray.DataArray 'latitude' ()>
array(-4.25, dtype=float32)
Coordinates:
    longitude  float32 15.25
    latitude   float32 -4.25
Attributes:
    standard_name:  latitude
    long_name:      latitude
    units:          degrees_north
    axis:           Y <xarray.DataArray 'longitude' ()>
array(15.25, dtype=float32)
Coordinates:
    longitude  float32 15.25
    latitude   float32 -4.25
Attributes:
    standard_name:  longitude
    long_name:      longitude
    units:          degrees_east
    axis:           X
<xarray.DataArray 'latitude' ()>
array(-4.25, dtype=float32)
Coordinates:
    longitude  float32 15.25
    latitude   float32 -4.25
Attributes:
    standard_name:  latitude
    long_name:      latitude
    units:          degrees_north
    axis:           Y <xarray.DataArray 'longitude' ()>
array(15.25, dtype=float32)
Coordinates:
    longitude  float32 15.25
    latitude   float32 -4.25
Attributes:
    standard_name:  longitude
    long_name:      longitude
    units:         

<xarray.DataArray 'latitude' ()>
array(-15.5, dtype=float32)
Coordinates:
    longitude  float32 28.25
    latitude   float32 -15.5
Attributes:
    standard_name:  latitude
    long_name:      latitude
    units:          degrees_north
    axis:           Y <xarray.DataArray 'longitude' ()>
array(28.25, dtype=float32)
Coordinates:
    longitude  float32 28.25
    latitude   float32 -15.5
Attributes:
    standard_name:  longitude
    long_name:      longitude
    units:          degrees_east
    axis:           X
<xarray.DataArray 'latitude' ()>
array(-15.5, dtype=float32)
Coordinates:
    longitude  float32 28.25
    latitude   float32 -15.5
Attributes:
    standard_name:  latitude
    long_name:      latitude
    units:          degrees_north
    axis:           Y <xarray.DataArray 'longitude' ()>
array(28.25, dtype=float32)
Coordinates:
    longitude  float32 28.25
    latitude   float32 -15.5
Attributes:
    standard_name:  longitude
    long_name:      longitude
    units:         

<xarray.DataArray 'latitude' ()>
array(-26., dtype=float32)
Coordinates:
    longitude  float32 32.5
    latitude   float32 -26.0
Attributes:
    standard_name:  latitude
    long_name:      latitude
    units:          degrees_north
    axis:           Y <xarray.DataArray 'longitude' ()>
array(32.5, dtype=float32)
Coordinates:
    longitude  float32 32.5
    latitude   float32 -26.0
Attributes:
    standard_name:  longitude
    long_name:      longitude
    units:          degrees_east
    axis:           X
<xarray.DataArray 'latitude' ()>
array(-26., dtype=float32)
Coordinates:
    longitude  float32 32.5
    latitude   float32 -26.0
Attributes:
    standard_name:  latitude
    long_name:      latitude
    units:          degrees_north
    axis:           Y <xarray.DataArray 'longitude' ()>
array(32.5, dtype=float32)
Coordinates:
    longitude  float32 32.5
    latitude   float32 -26.0
Attributes:
    standard_name:  longitude
    long_name:      longitude
    units:          degrees

<xarray.DataArray 'latitude' ()>
array(-20.25, dtype=float32)
Coordinates:
    longitude  float32 55.0
    latitude   float32 -20.25
Attributes:
    standard_name:  latitude
    long_name:      latitude
    units:          degrees_north
    axis:           Y <xarray.DataArray 'longitude' ()>
array(55., dtype=float32)
Coordinates:
    longitude  float32 55.0
    latitude   float32 -20.25
Attributes:
    standard_name:  longitude
    long_name:      longitude
    units:          degrees_east
    axis:           X
<xarray.DataArray 'latitude' ()>
array(-20.25, dtype=float32)
Coordinates:
    longitude  float32 55.0
    latitude   float32 -20.25
Attributes:
    standard_name:  latitude
    long_name:      latitude
    units:          degrees_north
    axis:           Y <xarray.DataArray 'longitude' ()>
array(55., dtype=float32)
Coordinates:
    longitude  float32 55.0
    latitude   float32 -20.25
Attributes:
    standard_name:  longitude
    long_name:      longitude
    units:          d

<xarray.DataArray 'latitude' ()>
array(-22.5, dtype=float32)
Coordinates:
    longitude  float32 17.0
    latitude   float32 -22.5
Attributes:
    standard_name:  latitude
    long_name:      latitude
    units:          degrees_north
    axis:           Y <xarray.DataArray 'longitude' ()>
array(17., dtype=float32)
Coordinates:
    longitude  float32 17.0
    latitude   float32 -22.5
Attributes:
    standard_name:  longitude
    long_name:      longitude
    units:          degrees_east
    axis:           X


## Ran

for city in cities_1:
    to_subset = station_coords[station_coords['station'] == city]
    to_subset = to_subset.reset_index()

    i=0
    for ifile in files_rea: 
        i=i+1
        reanalysis = {}
        reanalysis_keys = []
        var = 'pr'
        file_split=ifile.split("/")
        filename=file_split[-1]
        filename_split=filename.split("_")
        filename_base=filename.split(".")[0]
        institute=filename_split[2]  
        dataset=filename_split[3]
        #print([dataset, institute])
        file_xr = xr.open_dataset(ifile)
        key_name = 'results_'+dataset 
        reanalysis_keys.append(key_name)
        data_stations = get_station_point(file_xr=file_xr, station_coords=to_subset, var = var)
        reanalysis[key_name] = copy.deepcopy(data_stations)
        
    with open('data/pickles_py/point/cities/reanalysis_day_'+institute+'_'+dataset+'_'+city+'.pickle', 'wb') as handle:
        pickle.dump(reanalysis, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [ ]:
cities_1 = np.unique(station_coords.station)

In [ ]:
### Number of rainy days

fig=plt.figure(figsize=(20,12))
j=0
countries = cities_1

year1 = datetime(1991, 1, 1)
year2 = datetime(2020, 12, 31)

for country in countries:
    j=j+1
    ax = plt.subplot(4,6,j)
    pickles = glob.glob('data/pickles_py/point/cities/day/*'+country+'*pickle')
    pickles = sorted(pickles)
    with open(pickles[0], 'rb') as handle:
        a = pickle.load(handle)
    dataset = list(a.keys())[0]
    colnames = a[dataset].columns
    data = a[dataset]
    #times = rea_1.index
    months = ['J', 'F', 'M', 'A', 'M', 'J', 'J', 'A', 'S', 'O', 'N', 'D']

    ltypes_obs=['r--', 'r:', 'r-.', 'r:*']
    ltypes_rea=['k-', 'k:', 'b-.', 'r--', 'g--', 'k--', 'k-.', 'g-.']
    
    datasets = []
    i=0
    for pickle_1 in pickles:
        i=i+1
        with open(pickle_1, 'rb') as handle:
            b = pickle.load(handle)
        dataset = list(b.keys())[0]
        data = b[dataset]

        file_split=pickle_1.split("/")
        filename=file_split[-1]
        filename_split=filename.split("_")
        filename_base=filename.split(".")[0]
        institute=filename_split[1]  
        dataset=filename_split[3]
        
        data_1 = data[year1:year2]
        nyears= (year2.year-year1.year)
        data_1 = data_1[data_1>1].groupby(data_1.index.month).count()/nyears
        times = data_1.index
        ax.plot(times,data_1, ltypes_rea[i-1], linewidth=.9, label = dataset)
        ax.set_xticks(range(1,13))
        ax.set_xticklabels(months)   

        props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)

        textstr1 = country
        # place a text box in upper left in axes coords
        ax.text(0.05, 0.95, textstr1, fontsize=10, verticalalignment='top', transform=ax.transAxes, bbox=props)
  
        #all_together = pd.concat([all_together, data], 1)
    
        #altogether[country_all] = copy.deepcopy(all_together)        
plt.legend( bbox_to_anchor=(1.2, 1.0))
plt.suptitle('Seasonality of Rainy Days', y =0.92, size = 20)
plt.text(-62,20, 'Number of Rainy Days', rotation = 90, size = 18)
plt.text(-24,-6, 'Month', size = 18)
plt.show()

In [ ]:
### Seasonal Cycle

fig=plt.figure(figsize=(20,12))
j=0
countries = cities_1

year1 = datetime(1991, 1, 1)
year2 = datetime(2020, 12, 31)

for country in countries:
    j=j+1
    ax = plt.subplot(4,6,j)
    pickles = glob.glob('data/pickles_py/point/cities/day/*'+country+'*pickle')
    pickles = sorted(pickles)
    with open(pickles[0], 'rb') as handle:
        a = pickle.load(handle)
    dataset = list(a.keys())[0]
    colnames = a[dataset].columns
    data = a[dataset]
    #times = rea_1.index
    months = ['J', 'F', 'M', 'A', 'M', 'J', 'J', 'A', 'S', 'O', 'N', 'D']

    ltypes_obs=['r--', 'r:', 'r-.', 'r:*']
    ltypes_rea=['k-', 'k:', 'b-.', 'r--', 'g--', 'k--', 'k-.', 'g-.']
    
    datasets = []
    i=0
    for pickle_1 in pickles:
        i=i+1
        with open(pickle_1, 'rb') as handle:
            b = pickle.load(handle)
        dataset = list(b.keys())[0]
        data = b[dataset]

        file_split=pickle_1.split("/")
        filename=file_split[-1]
        filename_split=filename.split("_")
        filename_base=filename.split(".")[0]
        institute=filename_split[1]  
        dataset=filename_split[3]
        
        data_1 = data[year1:year2]
        nyears= (year2.year-year1.year)
        data_1 = data_1.groupby(data_1.index.month).sum()/nyears
        times = data_1.index
        ax.plot(times,data_1, ltypes_rea[i-1], linewidth=.9, label = dataset)
        ax.set_xticks(range(1,13))
        ax.set_xticklabels(months)   

        props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)

        textstr1 = country
        # place a text box in upper left in axes coords
        ax.text(0.05, 0.95, textstr1, fontsize=10, verticalalignment='top', transform=ax.transAxes, bbox=props)
  
        #all_together = pd.concat([all_together, data], 1)
    
        #altogether[country_all] = copy.deepcopy(all_together)        
plt.legend( bbox_to_anchor=(1.2, 1.0))
plt.suptitle('Monthly Climatology', y =0.92, size = 20)
plt.text(-62,250, 'Monthly Precipitation (mm)', rotation = 90, size = 18)
plt.text(-24,-60, 'Month', size = 18)
#plt.tight_layout()
plt.show()

In [ ]:
my_shp1 = 'data/Shapefiles/Southern_africa_countries_no_islands'
my_shp = 'data/Shapefiles/Southern_africa_countries_no_islands.shp'

to_clip = gpd.read_file(my_shp, crs="epsg:4326")

In [ ]:
fig=plt.figure(figsize=(20,12))
j=0
countries = cities_1

year1 = datetime(1991, 1, 1)
year2 = datetime(2020, 12, 31)

for country in countries:
    j=j+1
    ax = plt.subplot(4,6,j)
    pickles = glob.glob('data/pickles_py/point/cities/day/*'+country+'*pickle')
    #print(country)
    pickles = sorted(pickles)
    #print(pickles)
    with open(pickles[1], 'rb') as handle:
        a = pickle.load(handle)
    dataset = list(a.keys())[0]
    colnames = a[dataset].columns
    data = a[dataset]
    print(dataset)
    file_split=pickles[1].split("/")
    filename=file_split[-1]
    filename_split=filename.split("_")
    filename_base=filename.split(".")[0]
    institute=filename_split[1]  
    dataset=filename_split[3]
    #print(dataset)
    data_1 = data[year1:year2]
    nyears= (year2.year-year1.year)
    data_1 = data_1.resample('1M').sum()
    quants_ref = data_1.quantile(np.arange(0,1,.01))
    quants_ref_1 = quants_ref/(quants_ref.max()-quants_ref.min())
    quants_ref_new = data_1.quantile(np.arange(0,1.01,.25))
    
    stats = pd.DataFrame(quants_ref_new)
    stats.columns = [dataset]
    stats = stats.T
    ltypes_obs=['r--', 'r:', 'r-.', 'r:*']
    ltypes_rea=['k-', 'k:', 'b-.', 'r--', 'g--', 'k--', 'k-.', 'g-.']
    line_wd = [0.9, 2.5, 0.9, 0.9, 0.9, 0.9, 0.9, .9, .9, .9]
    
    datasets = []
    i=0
    for pickle_1 in pickles:
        i=i+1
        with open(pickle_1, 'rb') as handle:
            b = pickle.load(handle)
        dataset = list(b.keys())[0]
        data = b[dataset]

        file_split=pickle_1.split("/")
        filename=file_split[-1]
        filename_split=filename.split("_")
        filename_base=filename.split(".")[0]
        institute=filename_split[1]  
        dataset=filename_split[3]
        print(dataset)
        data_1 = data[year1:year2]
        nyears= (year2.year-year1.year)
        data_1 = data_1.resample('1M').sum()
        quants_obs = data_1.quantile(np.arange(0,1,.01))
        quants_obs_1 = quants_obs/(quants_ref.max()-quants_ref.min())
        quants_obs_new = data_1.quantile(np.arange(0,1.01,.25))
        quants_obs_new.columns = [dataset]
        stats = pd.concat([stats, pd.DataFrame(quants_obs_new).T], 0)
        
        ax.plot(quants_ref,quants_obs, ltypes_rea[i-1], linewidth=line_wd[i-1], label = dataset)
        #ax.set_xticks(range(1,13))
        #ax.set_xticklabels(months)   

        props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)

        textstr1 = country
        # place a text box in upper left in axes coords
        ax.text(0.05, 0.95, textstr1, fontsize=10, verticalalignment='top', transform=ax.transAxes, bbox=props)
  
        #all_together = pd.concat([all_together, data], 1)
    
        #altogether[country_all] = copy.deepcopy(all_together)        
plt.legend( bbox_to_anchor=(1.2, 1.0))
plt.suptitle('Monthly Quantile - Quantile Plot', y =0.92, size = 20)
plt.text(-1250,600, 'Dataset Quantiles (mm)', rotation = 90, size = 18)
plt.text(-500,-150, 'ARC2 Quantiles (mm)', size = 18)
#plt.tight_layout()
plt.show()

In [ ]:
pickles = glob.glob('data/pickles_py/point/cities/day/*Cape*pickle')
pickles = sorted(pickles)
with open(pickles[2], 'rb') as handle:
    a = pickle.load(handle)
dataset = list(a.keys())[0]
print(dataset)
data = a[dataset]

data.max()


In [ ]:
pickles

In [ ]:
fig=plt.figure(figsize=(20,12))
j=0
countries = cities_1

year1 = datetime(1991, 1, 1)
year2 = datetime(2020, 12, 31)

for country in countries:
    j=j+1
    ax = plt.subplot(4,6,j)
    pickles = glob.glob('data/pickles_py/point/cities/day/*'+country+'*pickle')
    #print(country)
    pickles = sorted(pickles)
    #print(pickles)
    with open(pickles[1], 'rb') as handle:
        a = pickle.load(handle)
    dataset = list(a.keys())[0]
    colnames = a[dataset].columns
    data = a[dataset]
    #print(dataset)
    file_split=pickles[1].split("/")
    filename=file_split[-1]
    filename_split=filename.split("_")
    filename_base=filename.split(".")[0]
    institute=filename_split[1]  
    dataset=filename_split[3]
    #print(dataset)
    data_1 = data[year1:year2]
    nyears= (year2.year-year1.year)
    data_1 = data_1.resample('1M').sum()
    quants_ref = data_1.quantile(np.arange(0,1,.01))

    ltypes_rea=['k-', 'k:', 'b-.', 'r--', 'g--', 'k--', 'k-.', 'g-.']
    line_wd = [0.9, 2.5, 0.9, 0.9, 0.9, 0.9, 0.9, .9, .9, .9]
    
    datasets = []
    i=0
    for pickle_1 in pickles:
        i=i+1
        with open(pickle_1, 'rb') as handle:
            b = pickle.load(handle)
        dataset = list(b.keys())[0]
        data = b[dataset]

        file_split=pickle_1.split("/")
        filename=file_split[-1]
        filename_split=filename.split("_")
        filename_base=filename.split(".")[0]
        institute=filename_split[1]  
        dataset=filename_split[3]
        #print(dataset)
        data_1 = data[year1:year2]
        nyears= (year2.year-year1.year)
        data_1 = data_1.resample('1M').sum()
        quants_obs = data_1.quantile(np.arange(0,1,.01))
        ax.plot(quants_ref,quants_obs, ltypes_rea[i-1], linewidth=line_wd[i-1], label = dataset)
        #ax.set_xticks(range(1,13))
        #ax.set_xticklabels(months)   

        props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)

        textstr1 = country
        # place a text box in upper left in axes coords
        ax.text(0.05, 0.95, textstr1, fontsize=10, verticalalignment='top', transform=ax.transAxes, bbox=props)
  
        #all_together = pd.concat([all_together, data], 1)
    
        #altogether[country_all] = copy.deepcopy(all_together)        
plt.legend( bbox_to_anchor=(1.2, 1.0))
plt.suptitle('Annual Quantile - Quantile Plot', y =0.92, size = 20)
plt.text(-1250,600, 'Dataset Quantiles (mm)', rotation = 90, size = 18)
plt.text(-500,-150, 'ARC2 Quantiles (mm)', size = 18)
#plt.tight_layout()
plt.show()

In [ ]:
cities_1 = np.unique(station_coords.station)
#cities_1 = ['Gaborone']
#fig=plt.figure(figsize=(15,12))
j=0
countries = cities_1

yr = '2010-07-01'
yr_2 = '2011-06-30'

year1 = datetime(2010, 7, 1)
year2 = datetime(2011, 6, 30)

rows, cols = 6, 4
fig, ax = plt.subplots(rows, cols,
                       sharex='col', figsize=(16,12))
for row in range(rows):
    for col in range(cols):
        ax1 = ax[row, col]
        
        j=j+1
        #print(j)
        
        if j==len(countries)+1:
            #print('jj')
            j=j-1
        #for country in countries:
        country = countries[j-1]
        #ax = plt.subplot(4,6,j)
        pickles = glob.glob('data/pickles_py/point/cities/day/*day*'+country+'*pickle')
        #print(country)
        pickles = sorted(pickles)
        ltypes_rea=['k-', 'k:', 'b-.', 'r--', 'g--', 'k--', 'k-.', 'g-.']
        i=0
        for pickle_1 in pickles:
            i=i+1
            reanalysis_keys = []

            with open(pickle_1, 'rb') as handle:
                b = pickle.load(handle)
            dataset = list(b.keys())[0]
            data = b[dataset]

            file_split=pickle_1.split("/")
            filename=file_split[-1]
            filename_split=filename.split("_")
            filename_base=filename.split(".")[0]
            institute=filename_split[2]  
            dataset=filename_split[3]
            #print(dataset)
            key_name = dataset
            reanalysis_keys.append(key_name)
            doy_stations = pd.DataFrame()
            df = data.loc[yr:yr_2]
            cum_sum = df.cumsum()
            ax1.plot(cum_sum, ltypes_rea[i-1], linewidth=.9, label = dataset)
            textstr1 = country
            if j == 13:
                ax1.set_ylabel('Precipitation (mm)', fontsize='xx-large')
            # place a text box in upper left in axes coords
            ax1.text(0.05, 0.95, textstr1, fontsize=10, verticalalignment='top', transform=ax1.transAxes, bbox=props)  
            #ax1.set_xticks( rotation = 90)
fig.autofmt_xdate(rotation=60)

fig.suptitle('Cummulative Precipitation'+'['+str(year1.year)+':'+str(year2.year)+']', fontsize='xx-large', y = 0.93)
#fig.ylabel('Figure suptitle', fontsize='xx-large')

plt.savefig('Images/cities_cumsum_'+str(year1.year)+'-'+str(year2.year)+'.png', bbox_inches='tight',dpi=300)

In [ ]:
year1

In [ ]:
seasons=["DJF","MAM","JJA","SON"]
for s in range(1,5):
    for key in regionstations.keys():
        fig=plt.figure(figsize=(20,2))
        for i,stcode in enumerate(regionstations[key]):
            pl=plt.subplot(1,5,i+1)
            ydata=stationdict[str(stcode)].resample("Q-Nov").sum()/10
            y=ydata[ydata.index.quarter==s]
            pl.plot(y)
            #pl.set_xlim(pd.to_datetime("1950-01-01"), pd.to_datetime("2020-12-31"))
            stname=metadata.Name.values[metadata.ID.values==stcode][0]
            pl.set_title(stname)
        plt.subplots_adjust(hspace=0.4, wspace=0.3)
        plt.suptitle(seasons[s-1]+" rainfall,"+key, y=1.1)
        plt.show()